[Sample dataset: recipes](https://docs.aiven.io/docs/products/opensearch/howto/sample-dataset)

In [1]:
# !pip install opensearch-py

In [2]:
import os,openai,json
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import HumanMessage

from opensearchpy import OpenSearch

In [3]:
SERVICE_URI = 'localhost:9200'
INDEX_NAME = 'epicurious-recipes'
os_client = OpenSearch(hosts=SERVICE_URI, ssl_enable=True)

In [4]:
INDEX_NAME = 'epicurious-recipes'
mapping_data = os_client.indices.get_mapping(INDEX_NAME)
mappings = mapping_data[INDEX_NAME]['mappings']
# print(json.dumps(mappings,indent=2))

In [5]:
DSL_TEMPLATE = """\
Given an input question, create a syntactically correct Elasticsearch query to run. \
Unless the user specifies in their question a specific number of examples they wish to obtain, \
always limit your query to at most {top_k} results. \
You can order the results by a relevant column to return the most interesting examples in the database.

Unless told to do not query for all the columns from a specific index, \
only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the mapping description. \
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which index. \
Return the query as valid json.

Use the following format:

Question: Question here
ESQuery: Elasticsearch Query formatted as json

Mapping: {mapping}

Question: {input}
ESQuery:
"""

In [6]:
def get_os_dsl(query,k):
    prompt = DSL_TEMPLATE.format(
        top_k=k,
        mapping=json.dumps(mappings,indent=2),
        input=query
    )
    chat = ChatOpenAI()
    messages = [HumanMessage(content=prompt)]
    response = chat(messages)
    # print(response.content)
    return response.content

In [7]:
import langchain; langchain.debug = True

In [8]:
top_k = 100
query = '2010年以降のレシピのうち、カロリーが高い順に100個のレシピのカロリーの散布図を書いてください。'
dsl = get_os_dsl(query,top_k)
print(dsl)

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Given an input question, create a syntactically correct Elasticsearch query to run. Unless the user specifies in their question a specific number of examples they wish to obtain, always limit your query to at most 100 results. You can order the results by a relevant column to return the most interesting examples in the database.\n\nUnless told to do not query for all the columns from a specific index, only ask for a the few relevant columns given the question.\n\nPay attention to use only the column names that you can see in the mapping description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which index. Return the query as valid json.\n\nUse the following format:\n\nQuestion: Question here\nESQuery: Elasticsearch Query formatted as json\n\nMapping: {\n  \"properties\": {\n    \"calories\": {\n      \"type\": \"float\"\n    },\n    \"categories\

[llm/end] [1:llm:ChatOpenAI] [3.11s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "{\n  \"size\": 100,\n  \"query\": {\n    \"range\": {\n      \"date\": {\n        \"gte\": \"2010-01-01\"\n      }\n    }\n  },\n  \"sort\": [\n    {\n      \"calories\": {\n        \"order\": \"desc\"\n      }\n    }\n  ],\n  \"_source\": [\"calories\"]\n}",
        "generation_info": {
          "finish_reason": "stop"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "{\n  \"size\": 100,\n  \"query\": {\n    \"range\": {\n      \"date\": {\n        \"gte\": \"2010-01-01\"\n      }\n    }\n  },\n  \"sort\": [\n    {\n      \"calories\": {\n        \"order\": \"desc\"\n      }\n    }\n  ],\n  \"_source\": [\"calories\"]\n}",
            "additional_kwargs": {}
       

In [9]:
res = os_client.search(index=INDEX_NAME,body=dsl)
print(json.dumps(res,indent=2))

{
  "took": 15,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 3948,
      "relation": "eq"
    },
    "max_score": null,
    "hits": [
      {
        "_index": "epicurious-recipes",
        "_id": "jJeRh4kBirRErZ0HS2YE",
        "_score": null,
        "_source": {
          "calories": 12213.0
        },
        "sort": [
          12213.0
        ]
      },
      {
        "_index": "epicurious-recipes",
        "_id": "XZeRh4kBirRErZ0HTnXs",
        "_score": null,
        "_source": {
          "calories": 11453.0
        },
        "sort": [
          11453.0
        ]
      },
      {
        "_index": "epicurious-recipes",
        "_id": "i5eRh4kBirRErZ0HTnbs",
        "_score": null,
        "_source": {
          "calories": 8858.0
        },
        "sort": [
          8858.0
        ]
      },
      {
        "_index": "epicurious-recipes",
        "_id": "q5eRh4kBir